In [119]:
# IMPORT HANDLER           PLEASE, KEEP IN MIND THAT THE IMPORTS HERE DO NOT CORRESPOND TO TOMAS' FILE IN THE GITHUB REPO. THE UPDATED FILES CAN BE FOUND IN MY LOCAL VERSION. LINK WILL BE PROVIDED

import os
import os.path
from datetime import datetime, timezone, timedelta

import cedalion
import cedalion.nirs
import cedalion.xrutils
import cedalion.xrutils as xrutils
from cedalion.datasets import get_fingertapping_snirf_path
from cedalion.dataclasses import PointType
import cedalion.plots

import re
import numpy as np
import pandas as pd
import xarray as xr
import pint
import pyxdf
import pyedflib
import pyvista as pv
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
from matplotlib.patches import Rectangle
import scipy.signal
from scipy import signal
from scipy.signal import find_peaks
from scipy.ndimage import uniform_filter1d
from scipy.signal import correlate

xr.set_options(display_max_rows=3, display_values_threshold=50)
np.set_printoptions(precision=4)
pv.set_jupyter_backend('server')

import sys
if '..' not in sys.path:
    sys.path.append('./../')

from data_loader import load_data         # MY DATA_LOADER FILE IS DIFFERENT TO TOMAS'
from data_loader import *
from headmodel import build_headmodel

from fnirs_preprocessing import prune_fnirs_channels
from utils_plots import plot_sci_psp_quality, plot_time_series

# To reaload packages and modules automatically! Super useful to avoid double coding
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [120]:
# PATH HANDLER

data_dir = '/Users/pavelsyarov/Desktop/CHARITE/Rotations/IBS/DOT_EEG/data/main'
sub = 'de341'
lsl_dir = os.path.join(data_dir, f'{sub}/{sub}.xdf')
snirf_path = os.path.join(data_dir, f'{sub}/aurora/2025-06-10_003.snirf')
capnograph_dir = os.path.join(data_dir, f'{sub}/capnograph')
nibp_dir = os.path.join(data_dir, f'{sub}/nibp')
fnirs_landmarks_pos_path = os.path.join(data_dir, f'{sub}/aurora/digpts.txt')  # From Aurora

In [121]:
# SNIRF LOADER

recordings = cedalion.io.read_snirf(snirf_path)
rec = recordings[0] # Here, I load the snirf recording container. rec['amp'] is the amplitude data xarray

In [122]:
# CAPNOGRAPH LOADER

gd_ds, cw_xr, pt_xr = load_lifesense_csvs(capnograph_dir)
co2 = cw_xr # For example, here I load CO2 data into an xarray with unix time

In [123]:
# NIBP LOADER

edf_file_path = f'{nibp_dir}/nibp_all_(1).edf'
edf = pyedflib.EdfReader(edf_file_path)
edf_info(edf)
ecg_1 = extract_edf_stream_from_open_file(edf, 'ECG 1') # For example, here I load ECG 1 data into an xarray with unix time
edf.close()
edf_stream_info(ecg_1)

Number of signals: 15
Signal labels: ['ECG 1', 'ECG 2', 'ECG 3', 'aVL', 'aVR', 'aVF', 'CPAP', 'Snore', 'Flow', 'SpO2', 'Pleth', 'Pulse', 'Activity', 'Position', 'Accu']
Annotations: (array([ 0.,  0., 78.]), array([-1., -1., -1.]), array(['Start', 'Lights on', 'RR: 118/92'], dtype='<U10'))


Label of signal: ECG 1
Sample rate: 256.0 Hz
Some samples: [901.0989 904.0293 906.9597 909.8901 912.8205 915.7509 918.6813 918.6813
 921.6117 924.5421 927.4725 927.4725 930.4029 933.3333 936.2637 936.2637
 939.1941 942.1245 942.1245 945.0549 945.0549 947.9853 947.9853 950.9158
 953.8462 953.8462 956.7766 956.7766 956.7766 959.707  959.707  962.6374
 962.6374 965.5678 965.5678 965.5678 968.4982 968.4982 968.4982 968.4982
 971.4286 971.4286 971.4286 974.359  974.359  974.359  974.359  974.359
 977.2894 977.2894 977.2894 977.2894 977.2894 977.2894 977.2894 977.2894
 977.2894 980.2198 980.2198 980.2198 980.2198 980.2198 980.2198 980.2198
 980.2198 980.2198 980.2198 980.2198 980.2198 977.2894 977.2894 97

Label of signal: Pleth
Sample rate: 128.0 Hz
Some samples: [526.0073 525.7631 525.0305 524.2979 523.3211 522.3443 521.3675 520.3907
 519.4139 518.4371 517.4603 516.4835 515.5067 513.5531 512.0879 510.6227
 508.9133 507.2039 506.2271 503.5409 500.1221 499.1453 496.7033 494.9939
 493.5287 492.0635 491.3309 489.6215 488.6447 487.6679 485.9585 484.7375
 483.5165 482.7839 482.0513 481.0745 479.3651 478.1441 477.4115 476.6789
 475.9463 475.2137 474.2369 473.2601 472.5275 472.2833 471.3065 470.3297
 469.5971 468.8645 468.6203 468.3761 468.1319 467.1551 466.9109 465.9341
 465.6899 465.6899 465.4457 464.4689 463.4921 462.5153 461.5385 460.5617
 458.8523 457.6313 456.8987 456.6545 456.4103 456.4103 457.3871 459.3407
 462.7595 466.1783 471.5507 478.3883 484.4933 492.0635 499.1453 505.2503
 511.1111 516.2393 520.6349 524.7863 528.4493 529.9145 531.3797 532.3565
 533.3333 534.3101 534.0659 534.3101 534.0659 534.3101 534.3101 534.3101
 534.3101 534.3101 534.0659 533.0891]


Label of signal: Pulse
Sa

In [124]:
# LSL LOADER (MARKERS, LIVEAMP, UNIX TIME)

streams, header = pyxdf.load_xdf(lsl_dir)

for stream in streams:
    name = stream['info']['name'][0]
    if name == 'PsychoPyMarker':
        markers_data, markers_ts, markers_metadata = read_stream(stream)
    if name == 'UnixTime_s':
        unix_time_data, unix_time_ts, unix_time_metadata = read_stream(stream)
    if name == 'LiveAmpSN-101410-1017':
            liveamp_data, liveamp_ts, liveamp_metadata = read_stream(stream)

        
liveamp_channels = read_eeg_metadata(liveamp_metadata)
electrodes_mne = build_eeg_mne(liveamp_data.T, liveamp_ts, liveamp_channels)
liveamp_aux_xr = build_liveamp_aux(liveamp_data.T, liveamp_ts, liveamp_channels)
markers_data = markers_to_pandas(markers_data, markers_ts)

first_unix_value = unix_time_data[0][0]
first_unix_lsl_ts = unix_time_ts[0]

markers = markers_data

print(f"Markers: {markers}\n")

prep_bhb = markers_data[markers_data['marker'] == 'prep_bhb']
prep_bhb_onset_lsl = prep_bhb['time'].iloc[0]
diff_between_begin_lsl_and_prep_bhb_onset = prep_bhb_onset_lsl - first_unix_lsl_ts
lsl_time_20_s_before_prep_bhb_onset = first_unix_lsl_ts + (diff_between_begin_lsl_and_prep_bhb_onset - 20)
how_much_to_add_to_unix_to_reach_20_s_before_prep_bhb_onset = lsl_time_20_s_before_prep_bhb_onset - first_unix_lsl_ts
unix_20_s_before_prep_bhb_onset = first_unix_value + how_much_to_add_to_unix_to_reach_20_s_before_prep_bhb_onset
print(f"Difference between beginning of LSL recording and onset of prep_bhb marker: {diff_between_begin_lsl_and_prep_bhb_onset}")
print(f"LSL starts recording at lsl time: {first_unix_lsl_ts} which is {first_unix_value} in unix time")
print(f"Prep_bhb marker onset comes {diff_between_begin_lsl_and_prep_bhb_onset} seconds later")
print(f"We want to retain the data from 20s before onset of the prep_bhb data, which means lsl time {lsl_time_20_s_before_prep_bhb_onset} and unix time {unix_20_s_before_prep_bhb_onset}")
t0_lsl_ts = lsl_time_20_s_before_prep_bhb_onset
t0_unix = unix_20_s_before_prep_bhb_onset
print(f"\n\n20s before prep_bhb onset in lsl time: {t0_lsl_ts}\n\n20s before prep_bhb onset in unix time: {t0_unix}")

prep_bhb_unix = unix_20_s_before_prep_bhb_onset + 20

Creating RawArray with float64 data, n_channels=44, n_times=864780
    Range : 186567147 ... 187431926 =  373128.719 ... 374858.251 secs
Ready.
Markers:                  marker           time
0              prep_bhb  373307.328131
1          bhb_interval  373313.546532
2           breath_hold  373323.550054
3         normal_breath  373338.557134
4           breath_hold  373373.548984
..                  ...            ...
115    walking_sham_off  374786.017718
116   walking_pinkie_on  374788.506422
117  walking_pinkie_off  374798.480128
118     walking_sham_on  374808.778433
119                      374812.447670

[120 rows x 2 columns]

Difference between beginning of LSL recording and onset of prep_bhb marker: 177.95048311632127
LSL starts recording at lsl time: 373129.3776479411 which is 1749574609.288153 in unix time
Prep_bhb marker onset comes 177.95048311632127 seconds later
We want to retain the data from 20s before onset of the prep_bhb data, which means lsl time 373287.3281310

In [125]:
print(f"This is the time of onset of prep_bhb (first PsychoPy marker) in unix time: {prep_bhb_unix}\n")
print(f"This is 20s before prep_bhb onset in unix time: {t0_unix}\nThat's what we will set as t0=0.0s for all modalities")

This is the time of onset of prep_bhb (first PsychoPy marker) in unix time: 1749574787.238636

This is 20s before prep_bhb onset in unix time: 1749574767.238636
That's what we will set as t0=0.0s for all modalities


In [126]:
print("As can be seen below, the stimulus onsets of only the second PsychoPy run were recorded. This will make the syncing tricky and one should look at marker onsets towards the middle of the experiment to use for syncing. \n")
print(rec.stim)

As can be seen below, the stimulus onsets of only the second PsychoPy run were recorded. This will make the syncing tricky and one should look at marker onsets towards the middle of the experiment to use for syncing. 

           onset  duration  value trial_type
0      37.982208      10.0    1.0         12
1      87.453696      10.0    1.0         12
2      89.075712      10.0    1.0         12
3      89.346048      10.0    1.0         12
4      89.616384      10.0    1.0         12
..           ...       ...    ...        ...
483  5054.337024      10.0    1.0         12
484  5069.340672      10.0    1.0         12
485  5104.349184      10.0    1.0         12
486  5119.352832      10.0    1.0         12
487  5154.361344      10.0    1.0         12

[488 rows x 4 columns]


In [127]:
print("We start with the CO2 xarray loaded from the capnograph looking like this:\n")
print(co2)
print(f"\nIf we convert the timestamps to Berlin time, we'll see that the capnograph started recording at 7AM ({co2.time.values[0]}). This is because of faulty settings. Next, we add 12 hours to the unix time manually\n")

co2.coords['time'] = co2.time + 43200

print("\nOur goal is then to only use the data from 20s before prep_bhb_onset\n")

start_time = t0_unix
co2_filtered = co2.sel(time=slice(start_time, None))

print(co2_filtered)
print(co2_filtered.time.values[0])

print("\nOur last step is to subtract t0_unix to get data starting from 0.0s")

co2_filtered.coords['time'] = co2_filtered.time - t0_unix

print(f"\nFirst CO2 timestamp in relative time: {co2_filtered.time.values[0]}")

We start with the CO2 xarray loaded from the capnograph looking like this:

<xarray.DataArray (time: 29949)> Size: 240kB
array([11., 19., 27., ..., 11., 15., 21.], shape=(29949,))
Coordinates:
  * time     (time) float64 240kB 1.75e+09 1.75e+09 ... 1.75e+09 1.75e+09

If we convert the timestamps to Berlin time, we'll see that the capnograph started recording at 7AM (1749531365.94). This is because of faulty settings. Next, we add 12 hours to the unix time manually


Our goal is then to only use the data from 20s before prep_bhb_onset

<xarray.DataArray (time: 29127)> Size: 233kB
array([ 3.,  3.,  3., ..., 11., 15., 21.], shape=(29127,))
Coordinates:
  * time     (time) float64 233kB 1.75e+09 1.75e+09 ... 1.75e+09 1.75e+09
1749574767.46

Our last step is to subtract t0_unix to get data starting from 0.0s

First CO2 timestamp in relative time: 0.22136402130126953


In [128]:
print(co2_filtered)

<xarray.DataArray (time: 29127)> Size: 233kB
array([ 3.,  3.,  3., ..., 11., 15., 21.], shape=(29127,))
Coordinates:
  * time     (time) float64 233kB 0.2214 0.4614 0.7114 ... 7.277e+03 7.277e+03


In [129]:
print(f"Now we want to do the same thing for the NIBP streams. Here is an example with ECG 1. It looks like this now:\n")
print(ecg_1)
print("\nFirst, we remove all data which occurs from before 20s of prep_bhb onset.\n")

ecg_1_filtered = ecg_1.sel(time=slice(start_time, None))

print("\nThen, we change from unix to relative time.\n")

ecg_1_filtered.coords['time'] = ecg_1_filtered.time - t0_unix

print(f"\nSo now, we get this xarray:\n{ecg_1_filtered}\nAnd this is the first timestamp:\n{ecg_1_filtered.time.values[0]} ")


Now we want to do the same thing for the NIBP streams. Here is an example with ECG 1. It looks like this now:

<xarray.DataArray 'ECG 1' (time: 2746880)> Size: 22MB
array([-4.9128e+03, -4.2799e+03, -5.6484e+03, ...,  1.4652e+00,
        1.4652e+00,  1.4652e+00], shape=(2746880,))
Coordinates:
  * time     (time) float64 22MB 1.75e+09 1.75e+09 ... 1.75e+09 1.75e+09
Attributes: (3/5)
    sampling_frequency:  256.0
    start_time_local:    2025-06-10 18:10:06
    ...                  ...
    description:         Physiological signal from ECG 1

First, we remove all data which occurs from before 20s of prep_bhb onset.


Then, we change from unix to relative time.


So now, we get this xarray:
<xarray.DataArray 'ECG 1' (time: 1988802)> Size: 16MB
array([13.1868, 45.4212, 98.1685, ...,  1.4652,  1.4652,  1.4652],
      shape=(1988802,))
Coordinates:
  * time     (time) float64 16MB 0.003551 0.007458 ... 7.769e+03 7.769e+03
Attributes: (3/5)
    sampling_frequency:  256.0
    start_time_local

In [130]:
print("Lastly, we want do this for the LSL streams. Here is an example of adjusting the PsychoPy markers stream to relative time:\n")
print(f"This is what the markers look like now. They have LSL time: {markers}\n")
print(f"We also now the value of t0 in LSL time from earlier: {t0_lsl_ts}\nSo we only need to subtract that value from the marker timestamps.\n")

markers['time'] = markers['time'] - t0_lsl_ts

print(f"Now, markers looks like this: \n{markers}")

Lastly, we want do this for the LSL streams. Here is an example of adjusting the PsychoPy markers stream to relative time:

This is what the markers look like now. They have LSL time:                  marker           time
0              prep_bhb  373307.328131
1          bhb_interval  373313.546532
2           breath_hold  373323.550054
3         normal_breath  373338.557134
4           breath_hold  373373.548984
..                  ...            ...
115    walking_sham_off  374786.017718
116   walking_pinkie_on  374788.506422
117  walking_pinkie_off  374798.480128
118     walking_sham_on  374808.778433
119                      374812.447670

[120 rows x 2 columns]

We also now the value of t0 in LSL time from earlier: 373287.32813105744
So we only need to subtract that value from the marker timestamps.

Now, markers looks like this: 
                 marker         time
0              prep_bhb    20.000000
1          bhb_interval    26.218401
2           breath_hold    36.221923
3  

In [131]:
print("Please keep in mind that there is a few seconds delay in the capnograph data and non-invasive bp data. A correction for this is yet to be implemented.")

Please keep in mind that there is a few seconds delay in the capnograph data and non-invasive bp data. A correction for this is yet to be implemented.
